### Two ways to create a CDF  

In [1]:
# 1st solution
from math import *
def CND(X):
    (a1,a2,a3,a4,a5)=(0.31938153,-0.356563782,1.781477937,-1.821255978,1.330274429)
    L = abs(X)
    K=1.0/(1.0+0.2316419*L)
    w=1.0-1.0/sqrt(2*pi)*exp(-L*L/2.)*(a1*K+a2*K*K+a3*pow(K,3)+a4*pow(K,4)+a5*pow(K,5))
    if X<0:
        w = 1.0-w
    return w
# print(CND(1))

# 3nd 

# 2nd solution
def phi(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0
'''The erf() function (error function at x) can be used to compute traditional statistical functions such as the CDF
In statistics, for nonnegative values of x, the error function has the following interpretation: 
for a random variable Y that is normally distributed with mean 0 and variance 0.5, erf(x) describes 
the probability of Y falling in the range [−x, x].'''
# phi(1)

'The erf() function (error function at x) can be used to compute traditional statistical functions such as the CDF\nIn statistics, for nonnegative values of x, the error function has the following interpretation: \nfor a random variable Y that is normally distributed with mean 0 and variance 0.5, erf(x) describes \nthe probability of Y falling in the range [−x, x].'

### Black Schole Merton model

In [2]:
from math import*
def BSM_price(S,K,t,vol,r,opt_type):
    d0 = log(S/(K*exp(-r*t)))*(1/(vol*sqrt(t)))-(0.5*vol*sqrt(t))
    d1 = d0 + vol*sqrt(t)
    if opt_type == "call" :
        BSM_price = S*phi(d1) - K*exp(-r*t)*phi(d0)
    elif opt_type == "put":
        BSM_price = K*exp(-r*t)*phi(-d0) - S*phi(-d1) 
    else :
        print("error")
    return BSM_price
# BSM_price(50,50,1,0.2,0.01,'call')

### Monte-carlo simulation GBM

In [12]:
import math
import numpy as np
# Vanilla option
def Monte_Carlo(S,K,t,vol,r,N_sim,n_path):
    dt = t/n_path ; payoff_call = 0
    for i in range(N_sim):
        x = S
        for j in range(n_path):
            norm = np.random.normal(0,1)
            x = x * math.exp((r - (vol**2)/2)*dt + vol*norm*math.sqrt(dt))
        payoff_call += max(x-K,0)
    payoff_call = payoff_call / N_sim
    return payoff_call*math.exp(-r*t)

Monte_Carlo(20,20,1,0.2,0.03,10000,200)

1.8711621838271286

# Heston

In [4]:
import math as m
def Heston_price(S,K,t,vol,r,volvol,theta,kappa,rho,n,M,opt_type):
    pr=[]
    dt=t/M
    s=np.zeros(M+1)
    v=np.zeros(M+1)
    pr=np.zeros(n+1)
    s[0]=S
    v[0]=vol
    for j in range(n):
        for i in range(1,M+1):
            dW1=np.random.normal()*m.sqrt(dt)
            dW2=rho*dW1 + m.sqrt(1-rho**2)*np.random.normal()*m.sqrt(dt)
            v[i]=v[i-1]+kappa*(theta-v[i-1])*dt+volvol*m.sqrt(v[i-1])*dW2
            s[i]=s[i-1]+s[i-1]*(r*dt+m.sqrt(v[i-1])*dW1)
        if opt_type=='call':
            pr[j]=max(s[M]-K,0)
        elif opt_type=='put':
            pr[j]=max(K-s[M],0)
        else:
            print('option type not valid. please enter ''call'' or ''put''')
    return np.mean(pr)*m.exp(-r*t)
            
print(Heston_price(100,100,1,0.15,0.03,0.15,0.125,1.05,-0.5,100,252,'call'))

15.360613449948126


### Extract implied volatility using Newton Raphson

In [5]:
def Implied_vol(market_p, S, K, t, r, opt_type):
    delta_x = 10^-10 
    EPS = 10^-5
    cur_x = 0.2
    for i in range(10): 
        fx = market_p - BSM_price(S,K,t,cur_x,r,opt_type)
        imp_vol = cur_x - delta_x
        fx_delta = market_p - BSM_price(S,K,t,imp_vol,r,opt_type)
        dx = (fx - fx_delta) / delta_x
        if abs(dx) < EPS :
            break
        cur_x = cur_x - (fx/dx)
    return(cur_x)
# Implied_vol(2.65,20,20,1,0.03,'call')

### Finite difference method

In [6]:
import numpy as np
def greek_finite_diff(function,*args):
    print('\033[1m'+'Spot price = 0 , Rate = 2, Time = 3, Volatility = 4, STOP = 99 \n'+'\033[0m')
    checkvarb=[0,2,3,4] ; dx = 0.000001 ; checkordr=[1,2]
    while True:
        varb=int(input('select your variable\n'))
        if varb in checkvarb:
            ordr=int(input('select the order of differentiation \n'))
            if ordr==1:
                print((function(*args[:varb],args[varb]+dx,*args[varb+1:])-function(*args))/dx)
            elif ordr==2:
                print((function(*args[:varb],args[varb]+dx,*args[varb+1:])-2*function(*args)+function(*args[:varb],args[varb]-dx,*args[varb+1:]))/dx**2)
            else:
                break
        else:
            break
    return 'DONE'
# print(greek_finite_diff(BSM_price,20,20,0.01,1,0.2,'call'))

### Pricer 3 in 1

In [28]:
import numpy as np
import pandas as pd
def pricer(function,*args):
    price = function(*args) ; greeks = [price] ; dx=0.000001
    for varb in range(5) : 
        if varb == 1 :
            gamma = (function(*args[:varb-1],args[varb-1]+dx,*args[varb:])-2*function(*args)+function(*args[:varb-1],args[varb-1]-dx,*args[varb:]))/dx**2
            greeks.append(gamma)
        else :
            greek = (function(*args[:varb],args[varb]+dx,*args[varb+1:])-function(*args))/dx
            greeks.append(greek)
    df = pd.DataFrame(greeks,index=('Price','Delta','Gamma','Theta','Vega','Rho'))
    df.columns = ["Pricing"]
    return df
pricer(BSM_price,20,20,1,0.2,0.03,'call')

,Pricing
Price,1.882681
Delta,0.598706
Gamma,0.088818
Theta,1.076079
Vega,7.733363
Rho,10.091460
